#README.txt

Drag and drop a ZIPPED version of the folder `/data` from GitHub repo into Colab.

< - - - - -

In [1]:
# then run this and refresh directory...

# unzip datasets
!unzip data.zip

Archive:  data.zip
   creating: data/job-postings/
  inflating: data/job-postings/Data_Job_NY.csv  
  inflating: data/job-postings/Data_Job_SF.csv  
  inflating: data/job-postings/Data_Job_TX.csv  
  inflating: data/job-postings/Data_Job_WA.csv  
  inflating: data/README.txt         
  inflating: data/resume.pdf         
   creating: data/resumes/
  inflating: data/resumes/kaggleResumes.csv  




---


# API

In [2]:
!pip install config

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import requests
import os
import config
# API Docs found here: https://developer.usajobs.gov/Tutorials/Search-Jobs

In [4]:
config.US_JOBS_API_KEY = "xVc0TZiLfhcr17ci7Ngk6bLAetdRVFgntm2pZgWNtww="
config.EMAIL_ADDRESS = "gjacobthomas@gmail.com"

Note: I'm assuming this API has some limiter on it so we don't want to lose access. -tyler

In [5]:
# more parameters are found here: https://developer.usajobs.gov/API-Reference/GET-api-Search
def queryJobsAPI(keyword, location):
  host = 'data.usajobs.gov' 
  # add these values in the config.py file
  userAgent = config.EMAIL_ADDRESS
  authKey = config.US_JOBS_API_KEY

  base_url = "https://data.usajobs.gov/api/search"

  parameters = {
      "Keyword": keyword,
      "LocationName": location
  }

  headers = {
      "Host": host,          
      "User-Agent": userAgent,          
      "Authorization-Key": authKey  
  }

  resp = requests.request("GET", base_url,headers=headers, params=parameters)
  result = resp.json()['SearchResult']['SearchResultItems']

  # get Job Title 
  print(result[1]['MatchedObjectDescriptor']['PositionTitle'])
  # get Job Summary
  print(result[1]['MatchedObjectDescriptor']['UserArea']['Details']['JobSummary'])
  return result




---


# KeyBERT Extraction Function

In [6]:
!pip install keybert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.0 MB/s 
     |████████████████████████████████| 237 kB 39.5 MB/s 
     |████████████████████████████████| 51 kB 5.8 MB/s 
     |████████████████████████████████| 5.8 MB 58.2 MB/s 
     |████████████████████████████████| 1.3 MB 67.4 MB/s 
     |████████████████████████████████| 182 kB 69.8 MB/s 
     |████████████████████████████████| 7.6 MB 35.5 MB/s 
  Created wheel for keybert: filename=keybert-0.7.0-py3-none-any.whl size=23800 sha256=7d03e4ee3a19d7307c78831aa1b1ed4b15cd46b88f3f8772a7fa19ca7d1b2208
  Stored in directory: /root/.cache/pip/wheels/6c/bc/8b/a51bee77aec33895e6c8c236144b4cc10875659c4d2c80f070
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=356242bd0129c9050436fb16cce045bac8fc828626bb85f0959fa9d37eae7279
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88

In [7]:
# Imports
from keybert import KeyBERT # pip install keybert (give it a minute...)

'''
/*---------------------------------------------------------------------
 |  Method: extractKeywordsBERT
 |
 |  Purpose: Uses the KeyBert Keyword Extraction Tool to extract
 |           and return keywords from a given corpus. 
 |      
 |  Author: Tyler Parks
 |  Created On: 10/30/22
 |
 |  Parameters:
 |      normalized_corpus -- A single string containing all text of the
 |                           normalized corpus.
 |
 |  Returns: 
 |      keywords -- List of collected keywords
 |      scores -- List of those keyword's scores
 |
 |  References: https://maartengr.github.io/KeyBERT/#usage
 |
 *-------------------------------------------------------------------*/
''' 
def extractKeywordsBERT(normalized_corpus):   
    print('---KeyBert Extraction---')
    print('------------------------\n')

    # init. language model 
    language_model = KeyBERT(model = 'all-mpnet-base-v2')

    # extract those keywords!
    data = language_model.extract_keywords( normalized_corpus, 
                                            keyphrase_ngram_range=(1, 3), 
                                            stop_words='english',
                                            use_maxsum=False, 
                                            use_mmr=True,
                                            diversity=0.7,
                                            nr_candidates=20, 
                                            top_n=15
                                        )

    # zip the lists
    zipped = list(map(list, zip(*data)))
    keywords = zipped[0]
    scores = zipped[1]

    print('-Skill-'.ljust(40), '-Score-')
    for i, value in enumerate(keywords):
        print(value.ljust(40), scores[i])
    print()

    return keywords, scores



---


# CSO-Classifier Extraction Function

In [8]:
!python -m spacy download en_core_web_sm
!pip install cso-classifier

2022-12-03 19:28:54.681641: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 5.2 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 1.3 MB/s 
     |████████████████████████████████| 50 kB 3.9 MB/s 
     |████████████████████████████████| 61 kB 8.5 MB/s 
     |████████████████████████████████| 1.5 MB 34.7 MB/s 
     |████████████████████████████████| 3.2 MB 45.1 MB/s 
     |████████████████████████████████| 23.4 MB 1.4 MB/s 
     |████████████████████████████████| 12.9 MB 10.7 MB/s 
     |████████████████████████████████| 45 kB 3.9 MB/s 
     |██████████████████

**Don't restart runtime if the terminal says so! Keep going.**

In [9]:
import nltk
nltk.download('stopwords')
# update spacy
!pip install -U spacy
import spacy
from cso_classifier import CSOClassifier      # import classifier tool

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.6 MB 2.3 MB/s 
     |████████████████████████████████| 819 kB 52.2 MB/s 
  Attempting uninstall: thinc
    Found existing installation: thinc 8.0.17
    Uninstalling thinc-8.0.17:
      Successfully uninstalled thinc-8.0.17
  Attempting uninstall: spacy
    Found existing installation: spacy 3.0.5
    Uninstalling spacy-3.0.5:
      Successfully uninstalled spacy-3.0.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cso-classifier 3.0 requires spacy==3.0.5, but you have spacy 3.4.3 which is incompatible.


In [10]:
# update the most recent model
CSOClassifier.update() 

# define the model object
CSO_Extractor = CSOClassifier(modules = "both", enhancement = "first", explanation = False)


# ======================================================
#     ONTOLOGY
# ======================================================
Updating the ontology file
[██████████████████████████████████████████████████] 7M/7M
[*] Done!
Extracting and converting ontology.
Creating ontology pickle file from a copy of the CSO Ontology found in /usr/local/lib/python3.8/dist-packages/cso_classifier/assets/cso.csv
Creating graph representation of the ontology.
Saving the graph representation of the ontology (in a pickle object) in /usr/local/lib/python3.8/dist-packages/cso_classifier/assets/cso_graph.p

# ======================================================
#     MODELS: CACHED & WORD2VEC
# ======================================================
Couldn't delete file cached model: not found
Couldn't delete file word2vec model: not found
Updating the models: cached and word2vec
[██████████████████████████████████████████████████] 63M/63M
[*] Done!
[██████████████████████████████████████████████████] 34

In [11]:
def extractKeywordsCSO(normalized_corpus):

  # run the extraction
  result = CSO_Extractor.run(normalized_corpus)

  print('-----CSO Extraction-----')
  print('------------------------\n')
  
  for keyword in result['union']:
    print(keyword)

  return result['union']

# **YAKE Extractor**

In [12]:
!pip3 install yake
import yake 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 60 kB 2.6 MB/s 
     |████████████████████████████████| 132 kB 10.3 MB/s 
  Created wheel for jellyfish: filename=jellyfish-0.9.0-cp38-cp38-linux_x86_64.whl size=70602 sha256=f2bc9f063b61b6325e2c9a1f42c790012ae15702bc2617f8253d18a78f4e0baf
  Stored in directory: /root/.cache/pip/wheels/f1/c7/3c/4c83132de76359e3a429fd09c08995945ca96c5290a41651d3
Successfully built jellyfish


In [22]:
def extractKeywordsYAKE(normalized_corpus):
    print()
    print('---YAKE Extraction---')
    print('------------------------\n')

    language = "en"
    max_ngram_size = 3
    deduplication_threshold = 0.9
    deduplication_algo = 'seqm'
    windowSize = 1
    numOfKeywords = 20

    kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords, features=None)
    keywords = kw_extractor.extract_keywords(normalized_corpus)

    zipped = list(map(list, zip(*keywords)))
    keywords = zipped[0]
    scores = zipped[1]

    print('-Skill-'.ljust(40), '-Score-')
    for i, value in enumerate(keywords):
        print(value.ljust(40), scores[i])
    
    return keywords, scores

# **Skill Matching**

In [46]:
!pip install gensim
!pip install jieba

from functools import reduce
import gensim
import nltk
from nltk.tokenize import word_tokenize
import jieba
nltk.download('punkt')
import re
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [65]:
import textwrap
wrapper = textwrap.TextWrapper(width=175)  # wrapper options

In [79]:
class SkillsMatching:
  def __init__(self, all_job_postings_keywords, all_jp_titles, all_resumes_keywords):
      self.all_job_postings_keywords = all_job_postings_keywords
      self.all_jp_titles = all_jp_titles
      self.all_resumes_keywords = all_resumes_keywords
      

  # output JSON with matching job postings for each resume
  def skill_match(self):
      results = []
      
      for resume_json in self.all_resumes_keywords:
          resume_arr = []
          for key in resume_json:
            # loop through the keys = every classifier 
            classifier_json = {key: {"matching_job": "",
                      "job_title": ""}}
            resume_keywords = resume_json[key]
            scores = self.sim_score(self.all_job_postings_keywords, resume_keywords)

            # grab largest score
            largestScore = max(scores)

            # grab the index of the largest score in the arr
            max_idx = np.argmax(scores)

            classifier_json[key]['matching_job'] = self.all_job_postings_keywords[max_idx]
            classifier_json[key]['job_title'] = self.all_jp_titles[max_idx]
            classifier_json[key]['score'] = largestScore

            resume_arr.append(classifier_json)
          results.append(resume_arr)
      return results


  def split_and_join_arr(self, arr):
    new_arr = []
    for w in arr:
      word_arr = re.split('\W+', w.lower())
      new_arr = new_arr + word_arr
    # print(new_arr)
    return new_arr
  
  def sim_score(self, docs, keywords):
    keywords = self.split_and_join_arr(keywords)
    gen_docs = [[w.lower() for w in word_tokenize(text)] 
            for text in docs]
    dictionary = gensim.corpora.Dictionary(gen_docs)
    corpus = [dictionary.doc2bow(gen_doc) for gen_doc in gen_docs]
    tf_idf = gensim.models.TfidfModel(corpus)
    sims = gensim.similarities.Similarity('/usr/workdir',tf_idf[corpus],
                                      num_features=len(dictionary))

    query_doc_bow = dictionary.doc2bow(keywords)
    query_doc_tf_idf = tf_idf[query_doc_bow]
    # print(sims[query_doc_tf_idf])
    return sims[query_doc_tf_idf]

  def print_results(self, match_job_postings):

    # init average scores
    avgScoreBERT = 0
    avgScoreCSO = 0
    avgScoreYAKE = 0

    # for each resume
    for i, resume in enumerate(match_job_postings):
      print('\n------------------------')
      print("Matching Job Posting for Resume %s" % (i + 1))
      for classifier_obj in resume:
        classifier_type = list(classifier_obj.keys())[0]

        # if BERT
        if classifier_type == 'keywordsBERT':
          avgScoreBERT += round(classifier_obj[classifier_type]['score'] * 100, 2)

        # else if CSO
        elif classifier_type == 'keywordsCSO':
          avgScoreCSO += round(classifier_obj[classifier_type]['score'] * 100, 2)

        # else YAKE
        else:
          avgScoreYAKE += round(classifier_obj[classifier_type]['score'] * 100, 2)

        print()
        print('Skill Extracted Using:', classifier_type)
        print('Matched JP Title:', classifier_obj[classifier_type]['job_title'])
        print('With a score of:', round(classifier_obj[classifier_type]['score'] * 100, 2), '%')

        # Wrap this text.
        word_list = wrapper.wrap(text = 
              'Matched JP Description: ' + classifier_obj[classifier_type]['matching_job'])

        # Print each line.
        for element in word_list:
            print(element)

    avgScoreBERT /= len(match_job_postings)
    avgScoreCSO /= len(match_job_postings)
    avgScoreYAKE /= len(match_job_postings)

    print()
    print('---POST PROCESSING ANALYSIS----')
    print('-------------------------------')
    print()
    print('Average Matching Scores for Skill Extraction Method')
    print('BERT: ', round(avgScoreBERT, 2), '%')
    print('CSO : ', round(avgScoreCSO, 2), '%')
    print('YAKE: ', round(avgScoreYAKE, 2), '%')



---


#Driver Code

In [87]:
# Imports
import pandas as pd     # pip install pandas. usage: loading data from csv files into dataframes
import nltk
from nltk.tokenize import word_tokenize  
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.utils import shuffle
nltk.download("all")

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Pac

True

In [86]:
### Helper Functions

# Function to retrieve text data.
# (either 1 or more job postings or resumes)?
def getFileData(filename, dir):
    return pd.read_csv('data/' + dir + '/' + filename)

# Function to normalize text data. 
# (some skill extraction tools will normalize text for us; however, if not, this function is here)
# includes removing stopwords, punctuation, dates, links, etc...
def normalizeCorpus(corpus):
    return corpus
    # need to fix this 
    # nltk_tokenList = word_tokenize(corpus)
    # lemmatizer = WordNetLemmatizer()
    # nltk_lemmaList = []
    # for word in nltk_tokenList:
    #     nltk_lemmaList.append(lemmatizer.lemmatize(word))

    # normalized_corpus = []  
    # nltk_stop_words = set(stopwords.words("english"))
    # for w in nltk_lemmaList:  
    #     if w not in nltk_stop_words:  
    #         normalized_corpus.append(w)

    # punctuation = ";:.,?!"
    # for word in normalized_corpus:
    #     if word in punctuation:
    #         normalized_corpus.remove(word)

    #still need to add dates and links
    # return normalized_corpus

# Function to extract skill words from a given corpus.
# ideally, this function will output a set of skills extracted from the corpus
def extractSkills(corpus):

    # run KeyBert Extraction
    keywordsBERT, scoresBERT = extractKeywordsBERT(corpus)

    # run CSO Classifier Extraction
    keywordsCSO              = extractKeywordsCSO(corpus)
    
    # extraction method 2
    keywordsYAKE, scoresYAKE = extractKeywordsYAKE(corpus)

    # extraction method n...

    # keep going!
    return {
              "keywordsBERT": keywordsBERT,
              "keywordsCSO": keywordsCSO,
              "keywordsYAKE": keywordsYAKE
           }

### Driver Code
if __name__ == '__main__':

    # fetch the data
    jpCorpus = []
    jpTitles = []

    # --- FETCH MORE DATA VIA API ---
    job_posting_obj = queryJobsAPI("Data Scientist", "Washington, DC")
    for i in range(len(job_posting_obj)):
      jpCorpus.append(job_posting_obj[i]['MatchedObjectDescriptor']['UserArea']['Details']['JobSummary'])
      jpTitles.append(job_posting_obj[i]['MatchedObjectDescriptor']['PositionTitle'])

    # --- FETCH DATA FROM /CONTENT/DATA/..


    # collect all Job Postings
    dfNY = getFileData('Data_Job_NY.csv', 'job-postings')
    dfSF = getFileData('Data_Job_SF.csv', 'job-postings')
    dfTX = getFileData('Data_Job_TX.csv', 'job-postings')
    dfWA = getFileData('Data_Job_WA.csv', 'job-postings')

    frames = [dfNY, dfSF, dfTX, dfWA] # build frame
    job_posting_dataframe = pd.concat(frames) # concat frames
    job_posting_dataframe = shuffle(job_posting_dataframe)  # shuffle df

    resume_dataframe      = getFileData('kaggleResumes.csv', 'resumes')

    jpCorpus += list(job_posting_dataframe['Job_Desc'])
    jpTitles += list(job_posting_dataframe['Job_title'])
    #---------------

    """
    # print the dataframes
    #print('DataFrame of Job Postings:')
    #print(job_posting_dataframe)    
    #print()

    #print('DataFrame of Resumes:')
    #print(resume_dataframe)
    #print()
    #----------------------
    """

    # fetch resumes by themselves
    rCorpus  = list(resume_dataframe['Resume'])
    #----------------------

    # Number of resume samples to view
    NUM_SAMPLES = 10
    # number of job postings to view
    NUM_JPS = 10

    print('\nCurrently Avaliable Job Postings: ', len(jpCorpus))
    print()

    # for each JOB POSTING from the corpus
    i = 0
    all_job_postings = []
    for posting in jpCorpus:
          
        # Wrap this text.
        word_list = wrapper.wrap(text=posting)
    
        print('Job Posting #', i+1)

        # Print each line.
        for element in word_list:
            print(element)    

        print()

        text = normalizeCorpus(posting)
        job_posting_json = extractSkills(text)
        all_job_postings.append(text)
        # print lines, we are done with this posting
        print('------------------------\n')

        # break, after X postings
        if i > NUM_JPS - 1:
          break

        i += 1

    #---------------------- 

    # for each RESUME from the corpus
    i = 0
    all_resumes_keywords = []
    for resume in rCorpus:

        # Wrap this text.
        word_list = wrapper.wrap(text = resume)
    
        print('Resume #', i+1)

        # Print each line.
        for element in word_list:
            print(element)    

        print()

        text = normalizeCorpus(resume)
        resume_json = extractSkills(text)
        all_resumes_keywords.append(resume_json)
        # print lines, we are done with this resume
        print('------------------------\n')

        # break, after X resumes
        if i > NUM_SAMPLES - 1:
            break

        i += 1
    #---------------------- 

    
# keep going!

# end of driver code
#---------------

Data Scientist
This position is located in the Department of Health & Human Services (HHS), Centers for Medicare & Medicaid Services (CMS), Office of Burden Reduction and Health Informatics (OBRHI), Emerging Innovations Group . As a Data Scientist, GS-1560-13, you will design, develop, and implement the analytical, statistical, and programming mechanisms necessary to collect, organize, analyze, and interpret unique and highly specialized data sets.

Currently Avaliable Job Postings:  925

Job Posting # 1
As a Data Scientist, you will provide data management and analytical support to the Technology Branch Office, Office of Public Trust, specific to the body worn camera and
electronic records management systems. This office was created to strengthen public trust in the National Park Service's law enforcement programs through the transparency,
availability, and accessibility of information. This is a fully remote position.

---KeyBert Extraction---
------------------------

-Skill-       

# Resume and Job Matching

In [88]:
# Resume and Job Posting Matching
matching = SkillsMatching(all_job_postings, jpTitles, all_resumes_keywords)
match_job_postings = matching.skill_match()
matching.print_results(match_job_postings)

#


------------------------
Matching Job Posting for Resume 1

Skill Extracted Using: keywordsBERT
Matched JP Title: Data Scientist
With a score of: 15.02 %
Matched JP Description: This is a Direct Hire Authority (DHA) solicitation utilizing the DHA for Certain Personnel of the DoD Workforce to recruit and appoint qualified
candidates to positions in the competitive service. About the Position: Put your mind to work! Do Data Science for complex analytical, mathematical, and statistical research
that shapes operational requirements and provide options on how to implement data driven solutions for the Department of Army.

Skill Extracted Using: keywordsCSO
Matched JP Title: Data Scientist
With a score of: 19.14 %
Matched JP Description: This position is located in the Department of Health & Human Services (HHS), Centers for Medicare & Medicaid Services (CMS), Office of Burden Reduction
and Health Informatics (OBRHI), Emerging Innovations Group . As a Data Scientist, GS-1560-13, you will de

# Resume Upload

Drag and Drop a Data Scientist resum into Colab 

Make sure the file is named **resume.pdf**

There is a dummy resume located in the `/data` directory.



< - - - - - - -

In [67]:
!pip install pdfminer3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [81]:
from pdfminer3.layout import LAParams, LTTextBox
from pdfminer3.pdfpage import PDFPage
from pdfminer3.pdfinterp import PDFResourceManager
from pdfminer3.pdfinterp import PDFPageInterpreter
from pdfminer3.converter import PDFPageAggregator
from pdfminer3.converter import TextConverter
import io
import sys
from contextlib import redirect_stdout
from io import StringIO 

In [82]:
resource_manager = PDFResourceManager()
fake_file_handle = io.StringIO()
converter = TextConverter(resource_manager, fake_file_handle, laparams=LAParams())
page_interpreter = PDFPageInterpreter(resource_manager, converter)

with open('/content/data/resume.pdf', 'rb') as fh:

    for page in PDFPage.get_pages(fh,
                                  caching=True,
                                  check_extractable=True):
        page_interpreter.process_page(page)

    text = fake_file_handle.getvalue()

# close open handles
converter.close()
fake_file_handle.close()

# normalize and extract skills 
text = normalizeCorpus(text)
resume_kw = extractSkills(text)

# Matching
matching = SkillsMatching(all_job_postings, jpTitles, [resume_kw])
match_job_postings = matching.skill_match()
matching.print_results(match_job_postings)

---KeyBert Extraction---
------------------------

-Skill-                                  -Score-
data scientist conento                   0.6736
naive bayes gradient                     0.2802
storm visualization                      0.2706
dynamodb                                 0.2613
path pagerank algorithms                 0.2317
monitoring features                      0.2183
churn mobile network                     0.2088
tableau pipeline combines                0.2003
forecasts based exponential              0.1974
3x                                       0.1743
juanjosecarin increased                  0.1656
matplotlib kaggle competition            0.1646
ratio students passing                   0.1619
gmms redefining job                      0.1464
housing cost fresh                       0.009

-----CSO Extraction-----
------------------------

association rules
knowledge discovery
communication networks
purchasing
logistic regression
engineers
sales
neural networks
netwo